In [2]:
import sqlite3
import pandas as pd
import seaborn as sns
import json
import numpy as np
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go


from sams.utils import load_data
from sams.config import datasets

2025-08-25 14:24:37.888 | INFO     | sams.config:<module>:15 - PROJ_ROOT path is: C:\Users\Admin\Documents\GitHub\sams
2025-08-25 14:24:37.935 | WARNING  | sams.config:<module>:83 - Google MAPS API key not found, using Nominatim geocoder
2025-08-25 14:24:37.941 | INFO     | sams.config:<module>:92 - Loaded 0 geocodes from cache


In [3]:
# Use the path from datasets metadata 
db_path = datasets["sams"]["path"]
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables:", [t[0] for t in tables])

cursor.close()
conn.close()

Tables: ['students', 'institutes']


In [ ]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

query = """
SELECT 
    module,
    academic_year,
    COUNT(*) AS count_students
FROM students
GROUP BY module, academic_year
ORDER BY module, academic_year;
"""

cursor.execute(query)
rows = cursor.fetchall()

for row in rows:
    print(f"Module: {row[0]}, Year: {row[1]}, Count: {row[2]}")

cursor.close()
conn.close()

Module: Diploma, Year: 2018, Count: 64218
Module: Diploma, Year: 2019, Count: 57824
Module: Diploma, Year: 2020, Count: 56600
Module: Diploma, Year: 2021, Count: 59278
Module: Diploma, Year: 2022, Count: 78196
Module: Diploma, Year: 2023, Count: 79700
Module: Diploma, Year: 2024, Count: 81789
Module: ITI, Year: 2017, Count: 35237
Module: ITI, Year: 2018, Count: 75211
Module: ITI, Year: 2019, Count: 64148
Module: ITI, Year: 2020, Count: 67415
Module: ITI, Year: 2021, Count: 81950
Module: ITI, Year: 2022, Count: 74104
Module: ITI, Year: 2023, Count: 92085
Module: ITI, Year: 2024, Count: 83958
Module: PDIS, Year: 2020, Count: 916
Module: PDIS, Year: 2021, Count: 874
Module: PDIS, Year: 2022, Count: 1187
Module: PDIS, Year: 2023, Count: 1287
Module: PDIS, Year: 2024, Count: 1453


In [4]:
conn = sqlite3.connect(db_path)

query = """
SELECT
    barcode,
    aadhar_no,
    gender,
    dob,
    annual_income,
    state,
    district,
    social_category,
    domicile,
    date_of_application,
    academic_year,
    year,
    course_name,
    course_period,
    reported_institute,
    reported_branch_or_trade,
    institute_district,
    type_of_institute,
    highest_qualification,
    gc,
    ph,
    es,
    sports,
    national_cadet_corps,
    orphan,
    ews,
    admission_status,
    enrollment_status,
    phase,
    option_data
FROM students
WHERE module = 'Diploma';
"""

diploma_df = pd.read_sql_query(query, conn)
conn.close()

diploma_df.head()


,barcode,aadhar_no,gender,dob,annual_income,state,district,social_category,domicile,date_of_application,...,ph,es,sports,national_cadet_corps,orphan,ews,admission_status,enrollment_status,phase,option_data
0,18P000173,yuHyzmPe2cGIN3TsA11RV51PLvfLYcoUGMmJFKP7xAI=,Male,30-Jan-2001,Between 2.5 To 8 lakh,Jharkhand,Other,NA,Outside Odisha State Applicant,11-Apr-2018,...,No,No,No,NA,NA,NA,No,No,2,[]
1,18P000202,AgPzPgZCw33tK6EvZqs8/KAXaALWLdyhMJzxGSUc/d0=,Male,12-Feb-2003,Upto 2.5 lakh,Odisha,Ganjam,Other,Odisha State Applicant,11-Apr-2018,...,No,No,No,A,NA,NA,No,No,2,[]
2,18P000215,2pJKsU1iZq9ipSiLbch3upa5qaTgxpU73tihGR5Bzy0=,Male,08-Feb-2002,Between 2.5 To 8 lakh,Odisha,Khurda,Other,Odisha State Applicant,12-Apr-2018,...,No,No,No,NA,NA,NA,No,No,2,"[{""Phase"": ""1"", ""Option_No"": ""1"", ""Institute_N..."
3,18P000283,qdWd2luTGhCcAHGzes9axW+7rnfpE3v+58IL32zTiwg=,Male,22-Oct-2001,Above 8 lakh,Tripura,Other,NA,Outside Odisha State Applicant,12-Apr-2018,...,No,No,No,NA,NA,NA,Yes,Yes,1,"[{""Phase"": ""1"", ""Option_No"": ""1"", ""Institute_N..."
4,18P000308,nZiMIc/6eGufdWeUY+NM3yofkeFn3CSxosQeGLyXSyw=,Male,19-Oct-2000,Between 2.5 To 8 lakh,Haryana,Other,NA,Outside Odisha State Applicant,12-Apr-2018,...,No,No,No,NA,NA,NA,No,No,2,[]


In [5]:
import json
import pandas as pd

# Function to parse JSON strings safely
def parse_option_data(val):
    """
    Safely parses a JSON string into a Python object.

    Parameters:
    ----------
    val : str, list, or any
        The value to be parsed. If it's a JSON string, it will be decoded.
        If it's already a list, it is returned unchanged.
        All other types return an empty list.

    Returns:
    -------
    list
        Parsed list from JSON string, or the original list, or an empty list if parsing fails.
    """
    if isinstance(val, str):
        try:
            return json.loads(val)
        except json.JSONDecodeError:
            return []
    elif isinstance(val, list):
        return val
    else:
        return []

# Apply to Diploma dataset
diploma_df['option_data'] = diploma_df['option_data'].apply(parse_option_data)

# Explode option_data so each option becomes its own row
diploma_exploded = diploma_df.explode('option_data').reset_index(drop=True)

# Flatten nested option_data dictionaries into separate columns
option_details = pd.json_normalize(diploma_exploded['option_data'])

# Combine the flattened option details with the rest of the data
diploma_df = pd.concat([diploma_exploded.drop(columns=['option_data']), option_details], axis=1)

diploma_df.head()


,barcode,aadhar_no,gender,dob,annual_income,state,district,social_category,domicile,date_of_application,...,orphan,ews,admission_status,enrollment_status,phase,Phase,Option_No,Institute_Name,Trade,Status
0,18P000173,yuHyzmPe2cGIN3TsA11RV51PLvfLYcoUGMmJFKP7xAI=,Male,30-Jan-2001,Between 2.5 To 8 lakh,Jharkhand,Other,NA,Outside Odisha State Applicant,11-Apr-2018,...,NA,NA,No,No,2,NaN,NaN,NaN,NaN,NaN
1,18P000202,AgPzPgZCw33tK6EvZqs8/KAXaALWLdyhMJzxGSUc/d0=,Male,12-Feb-2003,Upto 2.5 lakh,Odisha,Ganjam,Other,Odisha State Applicant,11-Apr-2018,...,NA,NA,No,No,2,NaN,NaN,NaN,NaN,NaN
2,18P000215,2pJKsU1iZq9ipSiLbch3upa5qaTgxpU73tihGR5Bzy0=,Male,08-Feb-2002,Between 2.5 To 8 lakh,Odisha,Khurda,Other,Odisha State Applicant,12-Apr-2018,...,NA,NA,No,No,2,1,1,"Govt. Polytechnic, Bhubaneswar",Computer Operator and Programming Assistant (N...,Opted
3,18P000215,2pJKsU1iZq9ipSiLbch3upa5qaTgxpU73tihGR5Bzy0=,Male,08-Feb-2002,Between 2.5 To 8 lakh,Odisha,Khurda,Other,Odisha State Applicant,12-Apr-2018,...,NA,NA,No,No,2,1,2,"Govt. Polytechnic, Bhubaneswar",Draughtsman (Mechanical) (NSQF) (Fresh),Opted
4,18P000215,2pJKsU1iZq9ipSiLbch3upa5qaTgxpU73tihGR5Bzy0=,Male,08-Feb-2002,Between 2.5 To 8 lakh,Odisha,Khurda,Other,Odisha State Applicant,12-Apr-2018,...,NA,NA,No,No,2,1,3,"Govt. Polytechnic, Bhubaneswar",Plumber (NSQF) (Fresh),Opted


In [ ]:
diploma_df.columns

Index(['barcode', 'aadhar_no', 'gender', 'dob', 'annual_income', 'state',
       'district', 'social_category', 'domicile', 'date_of_application',
       'academic_year', 'year', 'course_name', 'course_period',
       'reported_institute', 'reported_branch_or_trade', 'institute_district',
       'type_of_institute', 'highest_qualification', 'gc', 'ph', 'es',
       'sports', 'national_cadet_corps', 'orphan', 'ews', 'admission_status',
       'enrollment_status', 'Phase', 'Option_No', 'Institute_Name', 'Trade',
       'Status'],
      dtype='object')

In [7]:
diploma_df = diploma_df.rename(columns={
    'Phase': 'phase',
    'Option_No': 'option_no',
    'Institute_Name': 'institute_name',
    'Trade': 'trade',
    'Status': 'status'
})


In [8]:
# Ensure option_no is numeric
diploma_df['option_no'] = pd.to_numeric(diploma_df['option_no'], errors='coerce')

# Calculate number of applications as max option_no per barcode
diploma_df['num_applications'] = diploma_df.groupby('barcode')['option_no'].transform('max')

# Replace NaN with 0 and convert to integer
diploma_df['num_applications'] = diploma_df['num_applications'].fillna(0).astype(int)


In [9]:
diploma_df.columns

Index(['barcode', 'aadhar_no', 'gender', 'dob', 'annual_income', 'state',
       'district', 'social_category', 'domicile', 'date_of_application',
       'academic_year', 'year', 'course_name', 'course_period',
       'reported_institute', 'reported_branch_or_trade', 'institute_district',
       'type_of_institute', 'highest_qualification', 'gc', 'ph', 'es',
       'sports', 'national_cadet_corps', 'orphan', 'ews', 'admission_status',
       'enrollment_status', 'phase', 'option_no', 'institute_name', 'trade',
       'status', 'num_applications'],
      dtype='object')

In [14]:
# Ensure option_no is numeric
diploma_df['option_no'] = pd.to_numeric(diploma_df['option_no'], errors='coerce')

# Calculate what num_applications *should* be
diploma_df['num_applications_calc'] = diploma_df.groupby('barcode')['option_no'].transform('max')

# Filter mismatches but ignore cases where calculated value is 0 or NaN
mismatch_df = diploma_df[
    (diploma_df['num_applications'] != diploma_df['num_applications_calc']) &
    (diploma_df['num_applications_calc'].notna()) &
    (diploma_df['num_applications_calc'] != 0)
]

# Keep only relevant columns
mismatch_df = mismatch_df[['barcode', 'option_no', 'num_applications', 'num_applications_calc']]

# Optional: show unique barcodes count
print("Total mismatches (excluding missing option_no cases):", mismatch_df['barcode'].nunique())


Total mismatches (excluding missing option_no cases): 0


In [15]:
# save df in parquet format
from pathlib import Path
DATA_DIR = Path("C:/Users/Admin/Documents/GitHub/sams/data")
RAW_DATA_DIR = DATA_DIR / "raw"
output_path = Path(RAW_DATA_DIR) / "diploma_applications.pq"
diploma_df.to_parquet(output_path, index=False, compression='snappy')

In [ ]:
iti_app = pd.read_parquet(RAW_DATA_DIR / "iti_applications.pq", engine='pyarrow')
hss_app = pd.read_parquet(RAW_DATA_DIR / "hss_applications.pq", engine='pyarrow')

NameError: name 'pd' is not defined

In [22]:
hss_app.columns

Index(['aadhar_no', 'barcode', 'academic_year', 'reported_institute',
       'sams_code', 'stream', 'institute_district', 'institute_block',
       'type_of_institute', 'phase', 'year', 'admission_status', 'option_no',
       'num_applications'],
      dtype='object')